### Exercises

Using the repo setup directions, setup a new local and remote repository named spark-exercises. The local version of your repo should live inside of ~/codeup-data-science. This repo should be named spark-exercises

Save this work in your spark-exercises repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named spark101 for this exercise.

In [110]:
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql.functions import regexp_extract, regexp_replace, lit, round, col, mean, month, year ,quarter, when

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/23 11:19:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [4]:
#create panda dataframe
df = pd.DataFrame({"language":["python","sql","ruby","c","java","c++"]})
df

,language
0,python
1,sql
2,ruby
3,c
4,java
5,c++


In [5]:
#now create spark dataframe 
spark_df= spark.createDataFrame(df)
spark_df.show()

+--------+
|language|
+--------+
|  python|
|     sql|
|    ruby|
|       c|
|    java|
|     c++|
+--------+



In [6]:
#view schema
spark_df.printSchema()

root
 |-- language: string (nullable = true)



In [7]:
#shape of dataframe, not as simple as pandas
#import count
from pyspark.sql.functions import count, avg
spark_df.count(), len(spark_df.columns)

(6, 1)

In [8]:
#show first 5 records
spark_df.show(5)

+--------+
|language|
+--------+
|  python|
|     sql|
|    ruby|
|       c|
|    java|
+--------+
only showing top 5 rows



### 2.Load the mpg dataset as a spark dataframe.

In [9]:
#import data where mpg resides
from pydataset import data

In [10]:
#get mpg pyspark data
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)                         

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



#### a. Create 1 column of output that contains a message like the one below:
    The 1999 audi a4 has a 4 cylinder engine

In [11]:
from pyspark.sql.functions import concat
mpg.select(concat(lit("The "), "year", lit(" "), "manufacturer",lit(" "), "model", lit(" has a "), "cyl", lit(" cylinder engine")).alias("concat")).show(5,truncate = False)

+----------------------------------------+
|concat                                  |
+----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 6 cylinder engine|
+----------------------------------------+
only showing top 5 rows



#### b. Transform the trans column so that it only contains either manual or auto.


In [12]:
mpg.select("trans").show(1)

+--------+
|   trans|
+--------+
|auto(l5)|
+--------+
only showing top 1 row



In [13]:
mpg.select("trans",regexp_extract("trans" , r"(\w+)", 1).alias("transmission")).show(4)

+----------+------------+
|     trans|transmission|
+----------+------------+
|  auto(l5)|        auto|
|manual(m5)|      manual|
|manual(m6)|      manual|
|  auto(av)|        auto|
+----------+------------+
only showing top 4 rows



In [14]:
mpg.select(regexp_extract("trans" , r"(\w+)", 1).alias("transmission")).show(4)

+------------+
|transmission|
+------------+
|        auto|
|      manual|
|      manual|
|        auto|
+------------+
only showing top 4 rows



### 3. Load the tips dataset as a spark dataframe.

In [15]:
#load tips in spark dataframe
tips = spark.createDataFrame(data("tips"))
tips.show(3)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
+----------+----+------+------+---+------+----+
only showing top 3 rows



a.What percentage of observations are smokers?

In [16]:
tips.count()

244

In [17]:
#group by smoker and count
tips.groupby("smoker").count().show()

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [18]:
#now from groupby and count, make a new columne called percent using withColumn
tips.groupby("smoker").count().withColumn(
    "percent", round(col("count")/tips.count(),2)
    ).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|   0.62|
|   Yes|   93|   0.38|
+------+-----+-------+



b. Create a column that contains the tip percentage

In [19]:
tips.select(
    tips.tip,
     tips.total_bill, 
     round((tips.tip/tips.total_bill * 100),2)
    .alias("tip_percent")
    ).show(4)

+----+----------+-----------+
| tip|total_bill|tip_percent|
+----+----------+-----------+
|1.01|     16.99|       5.94|
|1.66|     10.34|      16.05|
| 3.5|     21.01|      16.66|
|3.31|     23.68|      13.98|
+----+----------+-----------+
only showing top 4 rows



c. Calculate the average tip percentage for each combination of sex and smoker.

In [22]:
tips.groupBy("sex","smoker").agg(
    round(mean(tips.tip/tips.total_bill),3).alias("avg_tip_percent")).show()

+------+------+---------------+
|   sex|smoker|avg_tip_percent|
+------+------+---------------+
|  Male|    No|          0.161|
|  Male|   Yes|          0.153|
|Female|    No|          0.157|
|Female|   Yes|          0.182|
+------+------+---------------+



### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below

In [23]:
from vega_datasets import data

In [24]:
weather = data.seattle_weather()
weather = spark.createDataFrame(weather)

In [25]:
weather.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



- Convert the temperatures to fahrenheit.


In [26]:
# c to f: (0°C × 9/5) + 32 = 32°F

In [27]:
#spark rewrites over column if same column name is created
weather = weather.withColumn(
    "temp_max", (weather.temp_max * 9/5 + 32)
        ).withColumn("temp_min", (weather.temp_min * 9/5 + 32))
weather.show(6)
    

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|   55.04|    41.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|   51.08|   37.04| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|   53.06|   44.96| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|   53.96|   42.08| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|   48.02|   37.04| 6.1|   rain|
|2012-01-06 00:00:00|          2.5|   39.92|   35.96| 2.2|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 6 rows



- Which month has the most rain, on average?


In [67]:
#create new column called month, groupby it and get mean of average precipitation
weather.withColumn(
    "month", month(weather.date)
    ).groupby(col("month")
             ).agg(mean(weather.precipitation).alias("avg_rainfall")
                  ).sort(col("avg_rainfall").desc()
                    ).first()
                

Row(month=11, avg_rainfall=5.354166666666667)

- Which year was the windiest?


In [72]:
#create new column called year, groupby it and get mean of average wind
weather.withColumn(
    "year", year(weather.date)
    ).groupby(col("year")
             ).agg(mean(weather.wind).alias("avg_wind")
                  ).sort(col("avg_wind").desc()
                        ).first()
        

Row(year=2012, avg_wind=3.400819672131147)

- What is the most frequent type of weather in January?


In [85]:
#filter by first month, group by weather and count weather event
weather.filter(
    month(weather.date) == 1
    ).groupby(weather.weather
             ).count(
                ).sort(
                    col("count").desc()
                    ).show()

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



- What is the average high and low temperature on sunny days in July in 2013 and 2014?


In [105]:
#filter on july,2012,2015 and sunny day and find average of max and min temp

(
weather.filter(month("date") == 7)
    .filter(year("date") > 2012)
    .filter(year("date") < 2015)
    .filter(col("weather") == lit("sun"))
    .agg(
        avg("temp_max").alias("average_high_temp"),
        avg("temp_min").alias("average_low_temp"),
    )
    .show()
)

+-----------------+-----------------+
|average_high_temp| average_low_temp|
+-----------------+-----------------+
|80.29192307692308|57.52884615384615|
+-----------------+-----------------+



- What percentage of days were rainy in q3 of 2015?

In [111]:
# filter year and 3rd quarter
(
    weather.filter(year('date') == 2015)
    .filter(quarter('date') == 3)
    .select(
        when(
            col('weather') == 'rain', 1
            ).otherwise(0).alias('rain'))
    .agg(mean('rain'))
    .show()
)

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



- For each year, find what percentage of days it rained (had non-zero precipitation).

In [114]:
#if any precipitation, set it to 1 and find average that days for each year
(
weather.withColumn(
    "year", year(weather.date)
    ).select(
        when(col("precipitation") > 0 , 1
            ).otherwise(0).alias("did_rain"),"year"
            ).groupby("year")
            .agg(mean("did_rain")).show()
)    

+----+-------------------+
|year|      avg(did_rain)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+

22/10/24 01:40:45 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 918028 ms exceeds timeout 120000 ms
22/10/24 01:40:45 WARN SparkContext: Killing executors is not supported by current scheduler.
